In [1]:
from pathlib import Path

In [2]:
video_dir=Path("/data/data/MSRVTT_Zero_Shot_QA/videos/all")
gt_file_question=Path("/data/data/MSRVTT_Zero_Shot_QA/test_q1000.json")
gt_file_answers=Path("/data/data/MSRVTT_Zero_Shot_QA/test_a.json")

In [3]:
import json

In [4]:
with gt_file_question.open("r") as f:
    # print(json.load(f))
    pass

with gt_file_answers.open("r") as f:
    # print(json.load(f))
    pass

In [5]:
import pandas as pd

In [6]:
q_df = pd.read_json(gt_file_question)
a_df = pd.read_json(gt_file_answers)

df = pd.merge(q_df, a_df, on="question_id")
df

,video_name,question_id,question,answer,type
0,video7121,v_video7121_5,what is middle aged women doing?,talk,3
1,video8431,v_video8431_6,what is a black colored car leading in in a mo...,race,3
2,video8183,v_video8183_10,who saludos amigos 1943 the three caballeros 1...,cartoon,3
3,video8504,v_video8504_15,who sits at a couch?,woman,3
4,video9285,v_video9285_12,what is a woman singing with horse in a stage?,song,3
...,...,...,...,...,...
995,video9497,v_video9497_29,what does a man sing about being possible whil...,anything,3
996,video7452,v_video7452_3,what is a man giving about his family?,interview,3
997,video9165,v_video9165_14,who plays electric guitar live on video?,man,3
998,video9320,v_video9320_21,who is laying next to a baby in a crib?,man,3


In [7]:
sample = df.iloc[0]
sample

video_name                            video7121
question_id                       v_video7121_5
question       what is middle aged women doing?
answer                                     talk
type                                          3
Name: 0, dtype: object

In [19]:
Path(video_dir).joinpath(f"{sample.video_name}.mp4").exists()

True

In [17]:
!ls /data/data/MSRVTT_Zero_Shot_QA/videos/all | grep 7121

video7121.mp4


In [43]:
import requests

# URL of the API endpoint; adjust localhost and port if different
url = "http://localhost:5000/predict"

samples = df.iloc[:10]

data = {"inputs": []}

for _, sample in samples.iterrows():
    data["inputs"].append(
        {
            "video_path": Path(video_dir)
            .joinpath(f"{sample.video_name}.mp4")
            .resolve()
            .as_posix(),
            "text_prompt": sample.question,
        }
    )

data

{'inputs': [{'video_path': '/data/data/MSRVTT_Zero_Shot_QA/videos/all/video7121.mp4',
   'text_prompt': 'what is middle aged women doing?'},
  {'video_path': '/data/data/MSRVTT_Zero_Shot_QA/videos/all/video8431.mp4',
   'text_prompt': 'what is a black colored car leading in in a motor racing competition?'},
  {'video_path': '/data/data/MSRVTT_Zero_Shot_QA/videos/all/video8183.mp4',
   'text_prompt': 'who saludos amigos 1943 the three caballeros 1945?'},
  {'video_path': '/data/data/MSRVTT_Zero_Shot_QA/videos/all/video8504.mp4',
   'text_prompt': 'who sits at a couch?'},
  {'video_path': '/data/data/MSRVTT_Zero_Shot_QA/videos/all/video9285.mp4',
   'text_prompt': 'what is a woman singing with horse in a stage?'},
  {'video_path': '/data/data/MSRVTT_Zero_Shot_QA/videos/all/video9486.mp4',
   'text_prompt': 'what do a overview of a females sim?'},
  {'video_path': '/data/data/MSRVTT_Zero_Shot_QA/videos/all/video8964.mp4',
   'text_prompt': 'what show?'},
  {'video_path': '/data/data/MSRVT

In [55]:
# Send a POST request to the server
response = requests.post(url, json=data)

# Check if the request was successful
if response.status_code == 200:
    result = response.json()
    print("Predicted text:", result["predicted_texts"])
else:
    print("Failed to get response:", response.status_code, response.text)

Failed to get response: 500 <!doctype html>
<html lang=en>
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>

